In [315]:
import os
import sys
os.chdir("/Users/DanPfeiffer/Documents/code/data-collector/src")
sys.path.append(os.getcwd())
import pandas as pd
from feature_eng import matchup_creator, box_score_feature_adder
import importlib
from datetime import datetime
from preprocess import preprocess_tr_stats, preprocess_box_scores
import json
import plotly.express as px

importlib.reload(matchup_creator)
importlib.reload(preprocess_tr_stats)
importlib.reload(preprocess_box_scores)
importlib.reload(box_score_feature_adder)


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [285]:
# stat_df_raw = pd.read_excel("../data/raw/full_stats.xlsx")

In [3]:
stat_df = stat_df_raw.copy()

In [4]:
stat_df.shape

(20128, 1589)

In [5]:
stat_df = preprocess_tr_stats.preprocess(stat_df)

In [6]:
stat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20128 entries, 0 to 20127
Columns: 215 entries, date to penalties_opponent_penalty_rate
dtypes: datetime64[ns](1), float64(213), object(1)
memory usage: 33.0+ MB


In [80]:
stat_df['date'] = stat_df['date'].dt.tz_localize('UTC')

In [8]:
box_df_raw = pd.read_csv("../data/raw/box_scores_df.csv")
box_df_raw.shape

(6265, 72)

In [316]:
box_df = box_df_raw.copy()

In [317]:
box_df = preprocess_box_scores.preprocess(box_df)

In [318]:
box_processed = box_df.copy()

In [319]:
bsfa = box_score_feature_adder.BoxScoreFeatureAdder(box_df=box_processed)

In [320]:
box_scores_df = bsfa.add_features(df=box_processed)

adding location info for venue
adding location info for home team
adding location info for away team
adding local time
[13.         12.         14.08333333 13.08333333 14.25       13.25
 20.5        13.5        17.         15.5        16.5        20.
 15.         18.5        21.         16.25       15.25       17.5
 19.5        19.         16.08333333 18.         15.08333333 12.5
 16.         20.25       19.25       17.25       18.25       17.08333333
 14.         16.28333333 14.5        16.75       18.46666667 19.33333333
 17.33333333 20.33333333 18.33333333 14.66666667 13.66666667 17.66666667
 18.41666667 19.41666667 21.25       17.48333333 20.41666667 18.48333333
 15.41666667 13.41666667 16.41666667 14.41666667 20.58333333 17.41666667
 16.58333333 20.16666667 15.66666667 13.58333333 19.16666667 16.33333333
 12.08333333 18.83333333 18.91666667 15.33333333 18.66666667 15.58333333
 18.16666667 20.91666667 16.66666667 18.08333333 20.08333333 17.83333333
 15.83333333]
adding travel dista

/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:125: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:125: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/Users/DanPfeiffer/Documents/code/data-collector/src/feature_eng/box_score_feature_adder.py:125: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

In [321]:
box_scores_df.shape

(4863, 256)

In [ ]:
for col in box_scores_df.columns:
    print(col)

In [85]:
post_processed_box_df= box_scores_df.copy().sort_values(by="date")

In [89]:
importlib.reload(matchup_creator)
mc = matchup_creator.MatchupCreator(stats_db=stat_df)

In [322]:
def generate_features(row, creator):
    game_id = row['id']
    home_short_display_name = row['home_short_display_name']
    away_short_display_name = row['away_short_display_name'] 
    date = row['date']
    features_df = creator.create_matchup(game_id=game_id, 
                                         home_team_box_short_display_name=home_short_display_name, 
                                         away_team_box_short_display_name=away_short_display_name, 
                                         date=date)
    return features_df.iloc[0]

In [ ]:
features_df = post_processed_box_df.apply(lambda row: generate_features(row, mc), axis=1)

creating matchup for 2019-01-05T21:35Z, Colts @ Texans
creating matchup for 2019-01-06T01:15Z, Seahawks @ Cowboys
creating matchup for 2019-01-06T18:05Z, Chargers @ Ravens
creating matchup for 2019-01-06T21:40Z, Eagles @ Bears
creating matchup for 2019-01-12T21:35Z, Colts @ Chiefs
creating matchup for 2019-01-13T01:15Z, Cowboys @ Rams
creating matchup for 2019-01-13T18:05Z, Chargers @ Patriots
creating matchup for 2019-01-13T21:40Z, Eagles @ Saints
creating matchup for 2019-01-20T20:05Z, Rams @ Saints
creating matchup for 2019-01-20T23:40Z, Patriots @ Chiefs
creating matchup for 2019-02-03T23:30Z, Patriots @ Rams
creating matchup for 2019-09-06T00:20Z, Packers @ Bears
creating matchup for 2019-09-08T17:00Z, Titans @ Browns
creating matchup for 2019-09-08T17:00Z, Ravens @ Dolphins
creating matchup for 2019-09-08T17:00Z, Falcons @ Vikings
creating matchup for 2019-09-08T17:00Z, Bills @ Jets
creating matchup for 2019-09-08T17:00Z, Washington @ Eagles
creating matchup for 2019-09-08T17:00Z

In [93]:
features_df.shape

(4863, 629)

In [95]:
df_final = pd.merge(post_processed_box_df, features_df, on='game_id', how='left')

In [96]:
df_final.shape

(4863, 885)

In [98]:
df_final.to_csv('../data/transformed/box_scores.csv', index=False)